# Build CLIP/SigLIP model and upload .mar to Google Cloud Storage

This notebook:

1. Downloads model from HuggingFace  
2. Saves model in *safe* format (safetensors)  
3. Creates `clip.mar`  
4. Uploads to your GCS bucket inside `model-store/clip.mar`  

You will then use this file in Cloud Run TorchServe.


In [ ]:
!pip install transformers safetensors google-cloud-storage torch-model-archiver


In [ ]:
BUCKET_NAME = "your-bucket-name"   # <-- change this
MODEL_NAME = "clip-vit-base-patch32"   # or "google/siglip-base"
EXPORT_DIR = "clip-model-safe"
MODEL_STORE_DIR = "model-store"

import os
os.makedirs(EXPORT_DIR, exist_ok=True)
os.makedirs(MODEL_STORE_DIR, exist_ok=True)


In [ ]:
from transformers import CLIPModel, CLIPProcessor

print("Downloading model:", MODEL_NAME)

model = CLIPModel.from_pretrained(f"openai/{MODEL_NAME}")
processor = CLIPProcessor.from_pretrained(f"openai/{MODEL_NAME}")

# ensure contiguous tensors (TorchServe requirement)
for name, param in model.named_parameters():
    param.data = param.data.contiguous()

model.save_pretrained(EXPORT_DIR, safe_serialization=True)
processor.save_pretrained(EXPORT_DIR)

print("Model saved to", EXPORT_DIR)


In [ ]:
!torch-model-archiver \
  --model-name clip \
  --version 1.0 \
  --serialized-file {EXPORT_DIR}/model.safetensors \
  --handler handler.py \
  --extra-files "{EXPORT_DIR}/config.json,{EXPORT_DIR}/preprocessor_config.json,{EXPORT_DIR}/tokenizer_config.json,{EXPORT_DIR}/merges.txt,{EXPORT_DIR}/vocab.json,{EXPORT_DIR}/special_tokens_map.json" \
  --export-path {MODEL_STORE_DIR} \
  --force

print("MAR created at:", f"{MODEL_STORE_DIR}/clip.mar")


In [ ]:
from google.cloud import storage

client = storage.Client()
bucket = client.bucket(BUCKET_NAME)

blob = bucket.blob("model-store/clip.mar")
blob.upload_from_filename(f"{MODEL_STORE_DIR}/clip.mar")

print("Uploaded to:")
print(f"gs://{BUCKET_NAME}/model-store/clip.mar")


In [ ]:
print("Use these ENV variables in Cloud Run services:")

print("\nTORCHSERVE_URL:")
print("https://<model-server-service>.run.app/predictions/clip")

print("\nMODEL_MAR_GCS_PATH:")
print(f"gs://{BUCKET_NAME}/model-store/clip.mar")
